<h3>Classe associada à análise exploratória do dataset</h3>

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import findspark
findspark.init()
import pyspark
import pandas as pd
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation
import numpy as np

In [2]:
class DataAnalysis:
    
    def __init__(self,df,spark):
        self.df = df
        self.spark = spark
        df.createOrReplaceTempView("table")
      
    def dropColNA(self,level):
        """Eliminação de colunas com percentagem de valores nulos maiores que level"""
        columns = self.getCols()
        percentage_of_na = []
        total_of_records = self.spark.sql("SELECT * FROM \
                            table").count()
        i = 0
        for col in columns:
            print(i, end='\r')
            col_na = self.spark.sql("SELECT "+ col +" FROM \
                            table WHERE "+ col +" IS NULL").count()
            col_percentage_of_na = round(col_na/total_of_records,3)
            percentage_of_na.append(col_percentage_of_na)
            i+=1
            if col_percentage_of_na > level:
                self.dropCol(col)
        fig, ax = plt.subplots(figsize=(13,14))
        return sns.barplot(x=percentage_of_na, y=columns)
    
            
    def stringToInt(self,cols):
        """Conversão de variáveis categóricas para númericas(Label Enconding / Ordinal Encoding)"""
        for col in cols:
            stringIndexer = StringIndexer(inputCol=col, outputCol=col+"_i")
            model = stringIndexer.fit(self.df)
            indexed = model.transform(self.df)
            self.df = indexed
            self.dropCol(col)
            self.df = self.df.withColumnRenamed(col+"_i",col)
        
        self.changeColDataTypes(cols)
        
    
    def removeMulticolinearity(self):
        """Remoção de colunas com multicolinearidade"""
        temp_df = self.df.select("*")
        cols = self.df.columns
        vec_assembler = VectorAssembler(inputCols = cols, outputCol="features")
        vec_df = vec_assembler.transform(temp_df)
        corr = Correlation.corr(vec_df, "features").collect()[0][0]
        corr = corr.toArray().tolist()
        corr_df = pd.DataFrame(data = corr, index = cols, columns = cols)
        del_col = []
        for col_1 in corr_df:
            for col_2 in cols:
                if col_1 != col_2 and (corr_df[col_1][col_2] > 0.7 or corr_df[col_1][col_2] < -0.7):
                    if col_1 not in del_col and col_2 not in del_col:
                        del_col.append(col_2)
        print(del_col)
        for col in del_col:
            self.dropCol(col)
    
        
    def dropValFromCol(self,col,val):
        """Remoção de registos que têm a coluna(col) com o valor(val)"""
        self.df = self.df.filter(self.df[col] != val)
    
    def dropCol(self,col):
        """Remoção de coluna(col)"""
        self.df = self.df.drop(col)
    
    def changeColDataTypes(self,col_types):
        """Conversão do tipo de dados de uma ou várias colunas"""
        for col in col_types:
            self.df =  self.df.withColumn(col, self.df[col].cast(col_types[col]))
        
    
    def getCountPlot(self,col):
        """Gráfico que serve de suporte à analise da distribuição dos valores de uma dada coluna(col)"""
        values = []
        for row in self.df.select(col).collect(): 
            values.append(row[col])
        sns.countplot(x=values)
        
    def dropRowNA(self):
        """Remoção de registos que têm pelo menos uma coluna com um valor omisso"""
        self.df = self.df.na.drop()
    
    def getCols(self):
        """Obtenção do esqueda das colunas do dataframe"""
        return self.df.schema.names  
    
    def getSpark(self):
        """Obtenção do núcleo spark associado à classe"""
        return self.spark
    
    def setdf(self,df):
        """Modificação do dataframe associado à classe"""
        self.df=df
        
    def getdf(self):
        """Obtenção do dataframe associado à classe"""
        return self.df
    
    def getRecordsCount(self):
        """Obtenção do total de registos presente no dataframe"""
        return self.df.count()
    
    
